## Testing

This notebook is designed to select the LLMs we will use for cross validation. I will consider the ten cheapest models on OpenRouter and assess their performance on the manually labelled data using the designed RAG pipeline.